# 語義搜尋

In [1]:
import openai
import re
import requests
import sys
import os
import pandas as pd
import numpy as np
from openai.embeddings_utils import get_embedding, cosine_similarity
from openai.embeddings_utils import distances_from_embeddings

In [ ]:
with open('secrets.toml', 'r') as f:
    config = toml.load(f)

openai.api_type = "azure"
openai.api_key = config['OPENAI_API_KEY']
openai.api_base = config['OPENAI_API_BASE']
openai.api_version = "2023-07-01-preview"

# 合約分析和生成

In [2]:
import openai
import re
import requests
import sys
import os
import pandas as pd
import numpy as np
from openai.embeddings_utils import get_embedding, cosine_similarity
from openai.embeddings_utils import distances_from_embeddings



## 關鍵條款提取

In [13]:
contract = """
This Contract for Services ("Agreement") is entered into as of [date], by and between Company A ("Company") and Company B ("Service Provider").
1.	Services Provided. Service Provider agrees to provide the following services to Company (the "Services"): The Service Provider agrees to provide consulting services to the Company in the field of marketing, including but not limited to market research, development of a marketing strategy, and implementation of marketing campaigns. The Service Provider shall provide reports and recommendations to the Company based on the results of the market research and the agreed-upon marketing strategy.
2.	Compensation. Company shall pay Service Provider the sum of 1.000.000 (One Million) $ for the Services. Payment shall be made on 15/9/2023.
3.	Term. This Agreement shall commence on 1/5/2023 and continue until 31/12/2023, unless earlier terminated by either party upon 30 days' prior written notice.
4.	Independent Contractor. Service Provider is an independent contractor, and nothing in this Agreement shall be construed as creating an employer-employee relationship, partnership, or joint venture between the parties.
5.	Confidentiality. Service Provider agrees to keep confidential any and all information learned or obtained as a result of providing the Services to Company. Service Provider shall not disclose such information to any third party without Company's prior written consent.
6.	Ownership of Work Product. Service Provider agrees that any and all work product produced in connection with the Services shall be the sole property of Company.
7.	Representations and Warranties. Service Provider represents and warrants that it has the necessary expertise and experience to perform the Services in a professional and workmanlike manner.
8.	Indemnification. Service Provider agrees to indemnify and hold harmless Company, its officers, directors, employees, and agents from and against any and all claims, damages, liabilities, costs, and expenses arising out of or in connection with the Services.
9.	Governing Law. This Agreement shall be governed by and construed in accordance with the laws of Italy without regard to conflicts of laws principles.
10.	Entire Agreement. This Agreement constitutes the entire agreement between the parties and supersedes all prior or contemporaneous negotiations, agreements, representations, and understandings between the parties, whether written or oral.
IN WITNESS WHEREOF, the parties have executed this Agreement as of the date first above written.
[Signature block for Company]
[Signature block for Service Provider]

"""

messages = [
    {"role": "system", "content": "You are a helpful assistant that has a deep understanding of contract. \
                                   You know all the detail about contract clauses.  \
                                   You will help people about their contract problems. \
                                   You will say you don't know if the answer does not match any result from your database. Be concise with your response. \
                                   Refrain from responding in simplified Chinese, you will respond in traditional Chinese at all time."},
]

# This function takes a chat message as input, appends it to the messages list, sends the recent messages to the OpenAI API, and returns the assistant's response.
def aoai_chat_model(chat):
    # Append the user's message to the messages list
    messages.append({"role": "user", "content": contract + chat})

    # Only send the last 5 messages to the API
    recent_messages = messages[-5:]

    # Send the recent messages to the OpenAI API and get the response
    response = openai.ChatCompletion.create(
      engine="gpt-4-32k",
      messages = recent_messages,
      temperature=0.7,
      max_tokens=800,
      top_p=0.95,
      frequency_penalty=0,
      presence_penalty=0,
      stop=None
      )

    # Append the assistant's response to the messages list
    messages.append({"role": "assistant", "content": response['choices'][0]['message']['content'].strip()})

    return response['choices'][0]['message']['content'].strip()

chat = input("Insert your questions: ")

aoai_chat_model(chat)

'公司應支付服務提供商1,000,000（一百萬）美元的服務費用。'

In [18]:
messages = [
    {"role": "system", "content": "You are a helpful assistant that has a deep understanding of contract. \
                                   You know all the detail about contract clauses.  \
                                   You will help people about their contract problems. \
                                   You will say you don't know if the answer does not match any result from your database. Be concise with your response. \
                                   Refrain from responding in simplified Chinese, you will respond in traditional Chinese at all time."},
]

def aoai_chat_model(chat, contract):
    # Append the user's message to the messages list
    messages.append({"role": "user", "content": contract + chat})

    # Only send the last 5 messages to the API
    recent_messages = messages[-5:]

    # Send the recent messages to the OpenAI API and get the response
    response = openai.ChatCompletion.create(
      engine="gpt-4-32k",
      messages=recent_messages,
      temperature=0.7,
      max_tokens=800,
      top_p=0.95,
      frequency_penalty=0,
      presence_penalty=0,
      stop=None
      )

    # Append the assistant's response to the messages list
    messages.append({"role": "assistant", "content": response['choices'][0]['message']['content'].strip()})

    return response['choices'][0]['message']['content'].strip()

def chat_conversation(contract):
    while True:
        chat = input("Insert your questions (or 'q' to quit): ")
        if chat.lower() == 'q':
            break
        response = aoai_chat_model(chat, contract)
        print('AI合約小幫手:', response)

contract = """
This Contract for Services ("Agreement") is entered into as of [date], by and between Company A ("Company") and Company B ("Service Provider").
1. Services Provided. Service Provider agrees to provide the following services to Company (the "Services"): The Service Provider agrees to provide consulting services to the Company in the field of marketing, including but not limited to market research, development of a marketing strategy, and implementation of marketing campaigns. The Service Provider shall provide reports and recommendations to the Company based on the results of the market research and the agreed-upon marketing strategy.
2. Compensation. Company shall pay Service Provider the sum of 1.000.000 (One Million) $ for the Services. Payment shall be made on 15/9/2023.
3. Term. This Agreement shall commence on 1/5/2023 and continue until 31/12/2023, unless earlier terminated by either party upon 30 days' prior written notice.
4. Independent Contractor. Service Provider is an independent contractor, and nothing in this Agreement shall be construed as creating an employer-employee relationship, partnership, or joint venture between the parties.
5. Confidentiality. Service Provider agrees to keep confidential any and all information learned or obtained as a result of providing the Services to Company. Service Provider shall not disclose such information to any third party without Company's prior written consent.
6. Ownership of Work Product. Service Provider agrees that any and all work product produced in connection with the Services shall be the sole property of Company.
7. Representations and Warranties. Service Provider represents and warrants that it has the necessary expertise and experience to perform the Services in a professional and workmanlike manner.
8. Indemnification. Service Provider agrees to indemnify and hold harmless Company, its officers, directors, employees, and agents from and against any and all claims, damages, liabilities, costs, and expenses arising out of or in connection with the Services.
9. Governing Law. This Agreement shall be governed by and construed in accordance with the laws of Italy without regard to conflicts of laws principles.
10. Entire Agreement. This Agreement constitutes the entire agreement between the parties and supersedes all prior or contemporaneous negotiations, agreements, representations, and understandings between the parties, whether written or oral.
IN WITNESS WHEREOF, the parties have executed this Agreement as of the date first above written.
[Signature block for Company]
[Signature block for Service Provider]

"""
chat_conversation(contract)


AI合約小幫手: 根據第二條，公司應支付服務提供者100萬美元作為服務費用。
AI合約小幫手: 根據第二條，付款的完成日期應為2023年9月15日。
AI合約小幫手: 合約的第八條是關於賠償的。服務提供者同意賠償並保護公司、其官員、董事、員工和代理人，免於因服務而產生或與之相關的任何和所有索賠、損害、責任、成本和費用。
AI合約小幫手: 此服務合同（“协议”）于[date]由A公司（“公司”）和B公司（“服务提供商”）签订。
1.	提供的服务。服务提供商同意向公司提供以下服务（“服务”）：服务提供商同意向公司提供营销咨询服务，包括但不限于市场研究、制定营销策略和实施营销活动。服务提供商应根据市场研究的结果和约定的营销策略向公司提供报告和建议。
2.	补偿。公司应支付服务提供商1,000,000（一百万）美元作为服务费。付款将在2023年9月15日进行。
3.	期限。本协议自2023年5月1日开始，持续至2023年12月31日，除非任何一方提前30天书面通知终止。
4.	独立承包商。服务提供商是独立承包商，本协议中的任何内容都不应被解释为在双方之间建立雇主-雇员关系、合伙关系或联营企业。
5.	保密。服务提供商同意对在向公司提供服务过程中学习或获得的所有信息保密。未经公司事先书面同意，服务提供商不得将此类信息透露给任何第三方。
6.	工作成果的所有权。服务提供商同意，与服务相关的所有工作成果应为公司的唯一财产。
7.	陈述和保证。服务提供商陈述并保证其具有以专业和精细的方式执行服务所必需的专业知识和经验。
8.	赔偿。服务提供商同意赔偿并使公司、其官员、董事、员工和代理人免受因服务产生或与之相关的任何和所有索赔、损害、责任、成本和费用的损害。
9.	管辖法律。本协议应根据意大利法律进行解释和执行，不考虑冲突法原则。
10.	完整协议。本协议构成双方之间的全部协议，取代双方之间的所有先前或同时进行的谈判、协议、陈述和理解，无论是书面还是口头。
兹证明，双方已于上述日期执行本协议。
[公司签名区]
[服务提供商签名区]
AI合約小幫手: 合同的第十條是关于整个协议。这个条款表明这个协议构成了双方之间的全部协议，并取代了双方之间所有先前或同时进行的谈判、协议、陈述和理解，无论是书面的还是口头的。


## 標記潛在問題

In [24]:
contract = """
This agreement ("Agreement") is made and entered into as of the date of execution by and between ABC Corporation ("ABC") and XYZ Company ("XYZ").

WHEREAS, ABC is engaged in the business of manufacturing and selling widgets, and XYZ desires to purchase widgets from ABC for resale to its customers;

NOW, THEREFORE, in consideration of the premises and mutual covenants herein contained, the parties agree as follows:

Sale of Products. ABC agrees to sell and deliver to XYZ and XYZ agrees to purchase and receive from ABC, widgets as specified in each purchase order issued by XYZ and accepted by ABC.

Price and Payment. The price for the widgets shall be as set forth in the applicable purchase order. Payment shall be due within 30 days of the invoice date.
Non-Competition. ABC agrees not to sell any products to any customers who have purchased products from our competitors within the past six months.
Warranty. ABC warrants that the widgets sold hereunder will conform to the specifications set forth in each purchase order and will be free from defects in material and workmanship for a period of one year from the date of delivery.
Limitation of Liability. ABC's liability for any breach of this Agreement or any warranty provided hereunder shall be limited to the purchase price of the widgets.
Governing Law. This Agreement shall be governed by and construed in accordance with the laws of the State of Delaware.

IN WITNESS WHEREOF, the parties have executed this Agreement as of the date first above written.

ABC Corporation
"""



# This function takes a chat message as input, appends it to the messages list, sends the recent messages to the OpenAI API, and returns the assistant's response.
def aoai_chat_model(chat):
    messages = [
    {"role": "system", "content": "You are a helpful assistant that has a deep understanding of contract. \
                                   You know all the detail about contract clauses.  \
                                   You will help people about their contract problems. \
                                   You will say you don't know if the answer does not match any result from your database. Be concise with your response. \
                                   Refrain from responding in simplified Chinese, you will respond in traditional Chinese at all time."},
]
    # Append the user's message to the messages list
    messages.append({"role": "user", "content": contract + chat})

    # Only send the last 5 messages to the API
    recent_messages = messages[-5:]

    # Send the recent messages to the OpenAI API and get the response
    response = openai.ChatCompletion.create(
      engine="gpt-4-32k",
      messages = recent_messages,
      temperature=0.7,
      max_tokens=800,
      top_p=0.95,
      frequency_penalty=0,
      presence_penalty=0,
      stop=None
      )

    # Append the assistant's response to the messages list
    messages.append({"role": "assistant", "content": response['choices'][0]['message']['content'].strip()})

    return response['choices'][0]['message']['content'].strip()

chat = "分析這份合同，並告訴我是否可能存在某些模糊或相互衝突的條款。"

aoai_chat_model(chat)


'從您提供的合同內容來看，以下是一些可能存在的問題：\n\n1. 非競爭條款: ABC同意不向過去六個月內從我們的競爭對手那裡購買產品的任何客戶銷售產品。這種語言可能會引起混淆，因為它並未明確指出誰是"我們的競爭對手"。此外，ABC如何知道客戶是否在過去六個月內購買了競爭對手的產品也是未明確的。\n\n2. 保證條款: ABC保證銷售的部件將符合每個採購訂單中的規範，並在交貨日期起一年內無材料和工藝缺陷。然而，這並未明確說明如果發生保證問題時，ABC需要採取什麼行動。\n\n3. 責任限制: ABC對本協議或此處提供的任何保證的違反的責任應限於部件的購買價格。這可能會導致問題，因為如果部件的缺陷導致了XYZ的損失超過部件的購買價格，那麼該條款可能會變得不公平。\n\n當然，對於這些可能的問題，您應該尋求專業法律建議，以確保您的權益得到保障。'

In [34]:
contract = """
This agreement ("Agreement") is made and entered into as of the date of execution by and between ABC Corporation ("ABC") and XYZ Company ("XYZ").

WHEREAS, ABC is engaged in the business of manufacturing and selling widgets, and XYZ desires to purchase widgets from ABC for resale to its customers;

NOW, THEREFORE, in consideration of the premises and mutual covenants herein contained, the parties agree as follows:

Sale of Products. ABC agrees to sell and deliver to XYZ and XYZ agrees to purchase and receive from ABC, widgets as specified in each purchase order issued by XYZ and accepted by ABC.

Price and Payment. The price for the widgets shall be as set forth in the applicable purchase order. Payment shall be due within 30 days of the invoice date.
Non-Competition. ABC agrees not to sell any products to any customers who have purchased products from our competitors within the past six months.
Warranty. ABC warrants that the widgets sold hereunder will conform to the specifications set forth in each purchase order and will be free from defects in material and workmanship for a period of one year from the date of delivery.
Limitation of Liability. ABC's liability for any breach of this Agreement or any warranty provided hereunder shall be limited to the purchase price of the widgets.
Governing Law. This Agreement shall be governed by and construed in accordance with the laws of the State of Delaware.

IN WITNESS WHEREOF, the parties have executed this Agreement as of the date first above written.

ABC Corporation
"""

def chat_conversation(contract):
    messages = [
    {"role": "system", "content": "You are a helpful assistant that has a deep understanding of contract. \
                                   You know all the detail about contract clauses.  \
                                   You will help people about their contract problems. \
                                   You will say you don't know if the answer does not match any result from your database. Be concise with your response. \
                                   Refrain from responding in simplified Chinese, you will respond in traditional Chinese at all time."},
]
    while True:
        chat = input("Insert your questions (or 'q' to quit): ")
        if chat.lower() == 'q':
            break

        # Append the user's message to the messages list
        messages.append({"role": "user", "content": contract + chat})

        # Only send the last 5 messages to the API
        recent_messages = messages[-5:]

        # Send the recent messages to the OpenAI API and get the response
        response = openai.ChatCompletion.create(
            engine="gpt-4-32k",
            messages=recent_messages,
            temperature=0.7,
            max_tokens=800,
            top_p=0.95,
            frequency_penalty=0,
            presence_penalty=0,
            stop=None
        )

        # Extract and clean the assistant's response
        assistant_response = response['choices'][0]['message']['content'].strip()
        cleaned_response = assistant_response.replace("\n\n", "\n")

        # Append the cleaned assistant's response to the messages list
        messages.append({"role": "assistant", "content": cleaned_response})
        
        print('You:', chat)
        print('AI合約小幫手:', cleaned_response)

chat_conversation(contract)

You: 你好
AI合約小幫手: 你好！請問你對這份合約有什麼問題或需要我解釋的地方？
You: 我想知道合約有沒有甚麼衝突的地方
AI合約小幫手: 根據你提供的合約內容，我並未發現明顯的衝突點。但請注意以下幾點：
1. 在非競爭條款中，ABC公司同意不向過去六個月內從競爭對手那裡購買產品的客戶銷售任何產品。這可能需要ABC公司進行額外的監控和驗證工作，以確保他們不違反該條款。
2. 在保證條款中，ABC公司保證其銷售的產品將符合每個採購單中設定的規格，並在交貨日期後一年內免於材料和工藝缺陷。這可能導致ABC公司承擔額外的風險和成本。
3. 在責任限制條款中，ABC公司對於任何違反此合約或此處提供的任何保證的責任將限制於產品的購買價格。這可能對XYZ公司不利，如果產品出現問題，XYZ公司可能無法得到足夠的賠償。
請確保你理解合約中的所有條款並與法律顧問諮詢，以確保你的權益受到保護。


In [25]:
contract = """
This agreement ("Agreement") is made and entered into as of the date of execution by and between ABC Corporation ("ABC") and XYZ Company ("XYZ").

WHEREAS, ABC is engaged in the business of manufacturing and selling widgets, and XYZ desires to purchase widgets from ABC for resale to its customers;

NOW, THEREFORE, in consideration of the premises and mutual covenants herein contained, the parties agree as follows:

Sale of Products. ABC agrees to sell and deliver to XYZ and XYZ agrees to purchase and receive from ABC, widgets as specified in each purchase order issued by XYZ and accepted by ABC.

Price and Payment. The price for the widgets shall be as set forth in the applicable purchase order. Payment shall be due within 30 days of the invoice date.
Non-Competition. ABC agrees not to sell any products to any customers who have purchased products from our competitors within the past six months.
Warranty. ABC warrants that the widgets sold hereunder will conform to the specifications set forth in each purchase order and will be free from defects in material and workmanship for a period of one year from the date of delivery.
Limitation of Liability. ABC's liability for any breach of this Agreement or any warranty provided hereunder shall be limited to the purchase price of the widgets.
Governing Law. This Agreement shall be governed by and construed in accordance with the laws of the State of Delaware.

IN WITNESS WHEREOF, the parties have executed this Agreement as of the date first above written.

ABC Corporation

"""

# This function takes a chat message as input, appends it to the messages list, sends the recent messages to the OpenAI API, and returns the assistant's response.
def aoai_chat_model(chat):
    messages = [
    {"role": "system", "content": "You are a helpful assistant that has a deep understanding of contract. \
                                   You know all the detail about contract clauses.  \
                                   You will help people about their contract problems. \
                                   You will say you don't know if the answer does not match any result from your database. Be concise with your response. \
                                   Refrain from responding in simplified Chinese, you will respond in traditional Chinese at all time."},
]
    # Append the user's message to the messages list
    messages.append({"role": "user", "content": contract + chat})

    # Only send the last 5 messages to the API
    recent_messages = messages[-5:]

    # Send the recent messages to the OpenAI API and get the response
    response = openai.ChatCompletion.create(
      engine="gpt-4-32k",
      messages = recent_messages,
      temperature=0.7,
      max_tokens=800,
      top_p=0.95,
      frequency_penalty=0,
      presence_penalty=0,
      stop=None
      )

    # Append the assistant's response to the messages list
    messages.append({"role": "assistant", "content": response['choices'][0]['message']['content'].strip()})

    return response['choices'][0]['message']['content'].strip()

chat = "分析這份合同，並告訴我是否有可能違反反壟斷法的條款。請明確標記出那些條款。"

aoai_chat_model(chat)


'這份合同中可能有可能違反反壟斷法的條款是"Non-Competition"。在此條款中，ABC同意不向任何在過去六個月內從競爭對手那裡購買產品的客戶銷售任何產品。這可能會被視為限制競爭的行為，因此可能違反反壟斷法。然而，這也取決於具體的司法管轄區以及相關的反壟斷法規定。'

## 提供合約模板

In [33]:
service_provider = "Supplier ABC"
client = "Company XYZ"
services_description = "installation, configuration, and maintenance of Company's IT infrastructure" 
start_date = "1/9/2023"
duration = "Three (3) years"
payment_terms = f"Within 30 days after receipt of an invoice from {service_provider}"
termination_notice = "30 days"
state = "Italy"

# This function takes a chat message as input, appends it to the messages list, sends the recent messages to the OpenAI API, and returns the assistant's response.
def aoai_chat_model(chat):
    messages = [
    {"role": "system", "content": "You are a helpful assistant that has a deep understanding of contract. \
                                   You know all the detail about contract clauses.  \
                                   You will help people about their contract problems. \
                                   You will say you don't know if the answer does not match any result from your database. Be concise with your response. \
                                   Refrain from responding in simplified Chinese, you will respond in traditional Chinese at all time. "
                                   },
]
    # Append the user's message to the messages list
    messages.append({"role": "user", "content": contract + chat})

    # Only send the last 5 messages to the API
    recent_messages = messages[-5:]

    # Send the recent messages to the OpenAI API and get the response
    response = openai.ChatCompletion.create(
      engine="gpt-4-32k",
      messages = recent_messages,
      temperature=0.7,
      max_tokens=800,
      top_p=0.95,
      frequency_penalty=0,
      presence_penalty=0,
      stop=None
      )

    # Extract, clean, and append the assistant's response
    assistant_response = response['choices'][0]['message']['content'].strip()
    cleaned_response = assistant_response.replace("\n", "")
    messages.append({"role": "assistant", "content": cleaned_response})

    return cleaned_response

chat = f"生成一份服務交付協議，包含以下元素：服務提供者：{service_provider}，客戶：{client}，服務描述：{services_description}，開始日期：{start_date}，期間：{duration}，付款條款：{payment_terms}，終止通知：{termination_notice}，州或國家：{state}。"

aoai_chat_model(chat)

'本協議（以下稱"協議"）於簽署日期由供應商ABC ("ABC") 與 Company XYZ ("XYZ") 共同簽訂。鑒於，ABC受聘於為XYZ的IT基礎設施進行安裝、配置和維護的業務，而XYZ希望從ABC購買這些服務；現在，考慮到前述原因和此處包含的相互義務，雙方同意如下：1. 服務提供：ABC同意根據每一份XYZ發出並由ABC接受的採購訂單，向XYZ提供並交付服務。2. 價格與付款：服務的價格將按照相應的採購訂單設定。付款應在收到發票之日起30天內完成。3. 服務期限：服務期限為自2023年9月1日起的三（3）年。4. 終止通知：任何一方都可以提前30天通知對方終止本協議。5. 法律適用：本協議將受意大利法律管轄並根據其解釋。證明以上事實，各方已於首次書面日期簽署此協議。ABC供應商'

## 語義分析

In [37]:
contract = """
This Carbon Capture and Storage Agreement ("Agreement") is made and entered into on 19/03/2023 by and between Company A, a corporation organized under the laws of France, 
and Company B, a corporation organized under the laws of Italy. The parties agree to collaborate on a CCS project to store 50 tons of CO2 in a saline aquifer located in Southern 
France using enhanced oil recovery (EOR) techniques. The parties will jointly design, construct, and operate the CCS facility, and will share the costs of the project in accordance 
with the proportion of their respective equity interests. Company A will provide the CO2 capture, compression, and transportation equipment, while Company B will provide the storage 
site and injection equipment. Both parties agree to comply with all applicable laws and regulations related to CCS, including the Environmental Protection Agency's (EPA) 
regulations on Underground Injection Control (UIC) Class VI wells.
"""

def chat_conversation(contract):
    messages = [
    {"role": "system", "content": "You are a helpful assistant that has a deep understanding of contract. \
                                   You know all the detail about contract clauses.  \
                                   You will help people about their contract problems. \
                                   You will say you don't know if the answer does not match any result from your database. Be concise with your response. \
                                   Refrain from responding in simplified Chinese, you will respond in traditional Chinese at all time."},
]
    while True:
        chat = input("Insert your questions (or 'q' to quit): ")
        if chat.lower() == 'q':
            break

        # Append the user's message to the messages list
        messages.append({"role": "user", "content": contract + chat})

        # Only send the last 5 messages to the API
        recent_messages = messages[-5:]

        # Send the recent messages to the OpenAI API and get the response
        response = openai.ChatCompletion.create(
            engine="gpt-4-32k",
            messages=recent_messages,
            temperature=0.7,
            max_tokens=800,
            top_p=0.95,
            frequency_penalty=0,
            presence_penalty=0,
            stop=None
        )

        # Extract and clean the assistant's response
        assistant_response = response['choices'][0]['message']['content'].strip()
        cleaned_response = assistant_response.replace("\n\n", "\n")

        # Append the cleaned assistant's response to the messages list
        messages.append({"role": "assistant", "content": cleaned_response})
        
        print('You:', chat)
        print('AI合約小幫手:', cleaned_response)

chat_conversation(contract)

You: 告訴我合約重點
AI合約小幫手: 這份"碳捕捉和儲存協議"是在2023年3月19日由法國的公司A和義大利的公司B簽訂的。雙方同意合作進行一個碳捕捉和儲存(CCS)項目，該項目將使用增強油田回收(EOR)技術在法國南部的一個鹽水含氣層中儲存50噸的二氧化碳。雙方將共同設計、建造和運營CCS設施，並將根據各自股權的比例分享項目的費用。公司A將提供二氧化碳的捕捉、壓縮和運輸設備，而公司B將提供儲存場地和注入設備。雙方都同意遵守所有與CCS相關的法律和規定，包括美國環保署(EPA)對地下注入控制(UIC)第六類井的規定。
You: EOR是哪一國採取的方案
AI合約小幫手: EOR，即增強油田回收，並非特定國家的方案。它是一種用於提高油田產量的技術，通過將氣體、水或化學物質注入油井，以將更多的石油從井中抽出。在這份協議中，EOR技術將用於將二氧化碳儲存在法國南部的一個鹽水含氣層中。
You: 合約是甚麼時候簽訂的
AI合約小幫手: 該合約是在2023年3月19日簽訂的。


# 客服中心分析

In [45]:
import openai
import re
import requests
import sys
import os
import pandas as pd
import numpy as np
import json


In [39]:
transcript = "Operator: Good morning, thank you for calling the auto insurance company, my name is John, how can I assist you today?\nCustomer: Yes, hi, I just noticed a dent on the side of my car and I have no idea how it got there. There were no witnesses around and I'm really frustrated.\nOperator: I'm sorry to hear that, I understand how frustrating it can be. Can you please provide me with your name and policy number so I can look up your account?\nCustomer: Yes, I’m Mario Rossi and the policy number is 123456.\nOperator: Thank you Mr. Rossi, let me take a look. I see that you've called earlier today, was there an issue with that call?\nCustomer: Yes, I was on hold for over an hour and the issue was not resolved. I'm really not happy about it.\nOperator: I'm sorry about that, let me assure you that we value your time and we'll do our best to assist you today. As for the dent on your car, I'd like to inform you that our policy does cover accidental damage like this. I can help you file a claim and connect you with one of our trusted repair shops in your area. Would you like me to proceed with that?\nCustomer: Yes, please. That would be great.\nOperator: Thank you for your cooperation. I'm now processing your claim and I'll be sending you an email with the next steps to follow. Please let me know if you have any other questions or concerns.\nCustomer: Thank you, I appreciate your help.\nOperator: You're welcome. Have a great day!\n\n\n"


# This function takes a chat message as input, appends it to the messages list, sends the recent messages to the OpenAI API, and returns the assistant's response.
def aoai_chat_model(chat):
    messages = [
    {"role": "system", "content": "You are a helpful assistant that has a deep understanding of contract. \
                                   You know all the detail about contract clauses.  \
                                   You will help people about their contract problems. \
                                   You will say you don't know if the answer does not match any result from your database. Be concise with your response. \
                                   Refrain from responding in simplified Chinese, you will respond in traditional Chinese at all time. "
                                   },
]
    # Append the user's message to the messages list
    messages.append({"role": "user", "content": transcript + chat})

    # Only send the last 5 messages to the API
    recent_messages = messages[-5:]

    # Send the recent messages to the OpenAI API and get the response
    response = openai.ChatCompletion.create(
      engine="gpt-4-32k",
      messages = recent_messages,
      temperature=0.7,
      max_tokens=800,
      top_p=0.95,
      frequency_penalty=0,
      presence_penalty=0,
      stop=None
      )

    # Extract, clean, and append the assistant's response
    assistant_response = response['choices'][0]['message']['content'].strip()
    cleaned_response = assistant_response.replace("\n", "")
    messages.append({"role": "assistant", "content": cleaned_response})

    return cleaned_response

chat = "從上述文本中提取以下信息：\n姓名和姓氏\n來電原因\n保單號碼\n解決方案, 輸出格式應為JSON。"

aoai_chat_model(chat)


'{"姓名和姓氏": "Mario Rossi","來電原因": "發現車子側面有凹痕，且不知道如何產生的","保單號碼": "123456","解決方案": "客服人員已經處理了他的索賠並將發送一封電子郵件告知他下一步該如何操作。他將被連接到他所在地區的一家信譽良好的修車店"}'

In [41]:
# This function takes a chat message as input, appends it to the messages list, sends the recent messages to the OpenAI API, and returns the assistant's response.
def aoai_chat_model(chat):
    messages = [
    {"role": "system", "content": "You are a helpful assistant that has a deep understanding of contract. \
                                   You know all the detail about contract clauses.  \
                                   You will help people about their contract problems. \
                                   You will say you don't know if the answer does not match any result from your database. Be concise with your response. \
                                   Refrain from responding in simplified Chinese, you will respond in traditional Chinese at all time. "
                                   },
]
    # Append the user's message to the messages list
    messages.append({"role": "user", "content": transcript + chat})

    # Only send the last 5 messages to the API
    recent_messages = messages[-5:]

    # Send the recent messages to the OpenAI API and get the response
    response = openai.ChatCompletion.create(
      engine="gpt-4-32k",
      messages = recent_messages,
      temperature=0.7,
      max_tokens=800,
      top_p=0.95,
      frequency_penalty=0,
      presence_penalty=0,
      stop=None
      )

    # Extract, clean, and append the assistant's response
    assistant_response = response['choices'][0]['message']['content'].strip()
    cleaned_response = assistant_response.replace("\n", "")
    messages.append({"role": "assistant", "content": cleaned_response})

    return cleaned_response

chat = "從對話中判斷起始和結尾的情感是什麼？"

aoai_chat_model(chat)

'對話的起始情感是挫折和困擾，因為顧客發現他的車有凹痕，並且他不知道如何造成的，同時他在之前的通話中等待了超過一個小時，問題仍未解決。然而，對話的結尾情感是感激和滿足，因為服務代表能夠幫助他處理索賠並提供修車服務的資訊，並且確保他的問題得到解決。'

In [55]:
# This function takes a chat message as input, appends it to the messages list, sends the recent messages to the OpenAI API, and returns the assistant's response.
def aoai_chat_model(chat):
    messages = [
    {"role": "system", "content": "You are a helpful assistant that has a deep understanding of contract. \
                                   You know all the detail about contract clauses.  \
                                   You will help people about their contract problems. \
                                   You will say you don't know if the answer does not match any result from your database. Be concise with your response. \
                                   Refrain from responding in simplified Chinese, you will respond in traditional Chinese at all time. "
                                   },
]
    # Append the user's message to the messages list
    messages.append({"role": "user", "content": chat})

    # Only send the last 5 messages to the API
    recent_messages = messages[-5:]

    # Send the recent messages to the OpenAI API and get the response
    response = openai.ChatCompletion.create(
      engine="gpt-4-32k",
      messages = recent_messages,
      temperature=0.7,
      max_tokens=800,
      top_p=0.95,
      frequency_penalty=0,
      presence_penalty=0,
      stop=None
      )

    # Extract, clean, and append the assistant's response
    assistant_response = response['choices'][0]['message']['content'].strip()
    messages.append({"role": "assistant", "content": assistant_response})
    return assistant_response

chat = transcript + "將上述文本分類為以下之一：報告問題、索賠處理或一般查詢。"

aoai_chat_model(chat)

'這段對話應該被分類為「索賠處理」。'

In [68]:
# This function takes a chat message as input, appends it to the messages list, sends the recent messages to the OpenAI API, and returns the assistant's response.
def aoai_chat_model(chat):
    messages = [
    {"role": "system", "content": "You are a helpful assistant that has a deep understanding of contract. \
                                   You know all the detail about contract clauses.  \
                                   You will help people about their contract problems. \
                                   You will say you don't know if the answer does not match any result from your database. Be concise with your response. \
                                   Refrain from responding in simplified Chinese, you will respond in traditional Chinese at all time. "
                                   },
]
    # Append the user's message to the messages list
    messages.append({"role": "user", "content": chat})

    # Only send the last 5 messages to the API
    recent_messages = messages[-5:]

    # Send the recent messages to the OpenAI API and get the response
    response = openai.ChatCompletion.create(
      engine="gpt-4-32k",
      messages = recent_messages,
      temperature=0.7,
      max_tokens=800,
      top_p=0.95,
      frequency_penalty=0,
      presence_penalty=0,
      stop=None
      )

    # Extract, clean, and append the assistant's response
    assistant_response = response['choices'][0]['message']['content'].strip()
    messages.append({"role": "assistant", "content": assistant_response})
    return assistant_response

chat = transcript + """
從上述文本中提取以下信息：\n客戶姓名\n來電原因\n保單號碼\n解決方案\n起始情感
\n起始情感的原因\n結尾情感\n結尾情感的原因\n聯絡中心改進的地方
\n類別(在報告問題、索賠處理和一般查詢中擇一)\n \n\n. 輸出格式應為JSON。
"""

aoai_chat_model(chat)



'{\n  "客戶姓名": "Mario Rossi",\n  "來電原因": "車子側面出現凹痕，並且在早些時候的通話未能解決問題",\n  "保單號碼": "123456",\n  "解決方案": "幫助客戶提出索賠，並將客戶連接到當地的信譽良好修理店",\n  "起始情感": "挫折",\n  "起始情感的原因": "車子受到損傷且先前的通話未能解決問題",\n  "結尾情感": "感謝",\n  "結尾情感的原因": "客服人員提供了解決方案並處理了他的索賠",\n  "聯絡中心改進的地方": "減少等待時間，提高問題解決效率",\n  "類別": "索賠處理"\n}'

In [70]:
print(aoai_chat_model(chat))

{
  "客戶姓名": "Mario Rossi",
  "來電原因": "車子側面出現凹痕，但不清楚原因，且之前的電話沒有解決問題",
  "保單號碼": "123456",
  "解決方案": "保險公司將處理索賠，並將客戶與當地的維修店聯繫",
  "起始情感": "挫折",
  "起始情感的原因": "車子出現凹痕且不知道原因，且之前致電未能解決問題",
  "結尾情感": "滿意",
  "結尾情感的原因": "保險公司將處理索賠，並將客戶與當地的維修店聯繫",
  "聯絡中心改進的地方": "減少客戶等待時間，提高問題解決效率",
  "類別": "索賠處理"
}


In [77]:
import json
with open('json_data.json', 'w', encoding='utf-8') as outfile:
    outfile.write(aoai_chat_model(chat))

# Opening JSON file with 'utf-8' encoding
with open('json_data.json', 'r', encoding='utf-8') as f:
    # Load the JSON object from the file
    data = json.load(f)

data

{'客戶姓名': 'Mario Rossi',
 '來電原因': '發現車子被凹傷，但不知道如何造成的，且之前的來電未解決問題',
 '保單號碼': '123456',
 '解決方案': '保險公司將處理索賠並連結客戶至附近的可信賴修車廠',
 '起始情感': '憤怒',
 '起始情感的原因': '車子受損且先前的通話未能解決問題',
 '結尾情感': '滿意',
 '結尾情感的原因': '保險公司同意處理索賠並提供解決方案',
 '聯絡中心改進的地方': '需要減少客戶等待時間，並在第一次通話中解決問題',
 '類別': '索賠處理'}

In [87]:
messages = [
{"role": "system", "content": "You are a helpful assistant that has a deep understanding of contract. \
                                You know all the detail about contract clauses.  \
                                You will help people about their contract problems. \
                                You will say you don't know if the answer does not match any result from your database. Be concise with your response. \
                                Refrain from responding in simplified Chinese, you will respond in traditional Chinese at all time. "
                                },
]

# Opening JSON file with 'utf-8' encoding
with open('json_data.json', 'r', encoding='utf-8') as f:
    # Load the JSON object from the file
    load_data = json.load(f)
    data = json.dumps(load_data, ensure_ascii=False, indent=4)

transcript = "Operator: Good morning, thank you for calling the auto insurance company, my name is John, how can I assist you today?\nCustomer: Yes, hi, I just noticed a dent on the side of my car and I have no idea how it got there. There were no witnesses around and I'm really frustrated.\nOperator: I'm sorry to hear that, I understand how frustrating it can be. Can you please provide me with your name and policy number so I can look up your account?\nCustomer: Yes, I’m Mario Rossi and the policy number is 123456.\nOperator: Thank you Mr. Rossi, let me take a look. I see that you've called earlier today, was there an issue with that call?\nCustomer: Yes, I was on hold for over an hour and the issue was not resolved. I'm really not happy about it.\nOperator: I'm sorry about that, let me assure you that we value your time and we'll do our best to assist you today. As for the dent on your car, I'd like to inform you that our policy does cover accidental damage like this. I can help you file a claim and connect you with one of our trusted repair shops in your area. Would you like me to proceed with that?\nCustomer: Yes, please. That would be great.\nOperator: Thank you for your cooperation. I'm now processing your claim and I'll be sending you an email with the next steps to follow. Please let me know if you have any other questions or concerns.\nCustomer: Thank you, I appreciate your help.\nOperator: You're welcome. Have a great day!\n\n\n"

def aoai_chat_model(chat):

    # Append the user's message to the messages list
    messages.append({"role": "user", "content": data + chat })

    # Only send the last 5 messages to the API
    recent_messages = messages[-5:]

    # Send the recent messages to the OpenAI API and get the response
    response = openai.ChatCompletion.create(
      engine="gpt-4-32k",
      messages = recent_messages,
      temperature=0.7,
      max_tokens=800,
      top_p=0.95,
      frequency_penalty=0,
      presence_penalty=0,
      stop=None
      )

    # Extract, clean, and append the assistant's response
    assistant_response = response['choices'][0]['message']['content'].strip()
    cleaned_response = assistant_response.replace("\n\n", "  ")
    messages.append({"role": "assistant", "content": cleaned_response})
    return cleaned_response



chat = "生成一封回應上述筆錄的電子郵件，通知客戶已創建了工單，並且如果是投訴，則表示歉意。客戶的名字是 {data['客戶姓名']}。"

aoai_chat_model(chat)

'親愛的 Mario Rossi 先生，  感謝您與我們聯繫，並向我們表達您的疑慮。我們深感抱歉，您的車輛有凹痕，並且先前的通話未能解決您的問題。  我們已經為您的問題創建了工單，並將由我們的索賠部門進行處理。我們會確保您的索賠過程順利進行，並會將您連接至附近的可信賴修車廠，以解決您車輛的問題。  我們明白您的困擾，並深感歉意，因為我們的服務未達到您的期待。我們會努力改善我們的服務，以減少客戶等待時間，並在第一次通話中解決問題。  我們重視您的反饋，並會努力提升我們的服務品質，以滿足您的需求。如果您有其他問題或疑慮，請隨時與我們聯繫。  謝謝您的理解與耐心。  順祝商祺，\n[您的名字]\n[您的職位]\n[聯絡資訊]'